In [24]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [25]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [26]:
# Store the CSV you saved created in part one into a DataFrame.
vacation_data_df = pd.read_csv("data/WeatherPy_vacation.csv")
vacation_data_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,0,Whitehorse,CA,69.80,60.72,-135.05,Westmark Hotel
1,1,Sao Gabriel Da Cachoeira,BR,75.20,-0.13,-67.09,Alarne - Área De Lazer Do Alto Rio Negro
2,2,Kapaa,US,78.80,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
3,3,Kindu,CD,70.65,-2.95,25.95,Hotel Karibu
4,4,Bairiki,KI,87.80,1.33,172.98,Tarawa Boutique Hotel


In [28]:
vacation_data_br = vacation_data_df.loc[vacation_data_df['Country'] == 'BR']
vacation_data_br.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
1,1,Sao Gabriel Da Cachoeira,BR,75.20,-0.13,-67.09,Alarne - Área De Lazer Do Alto Rio Negro
28,28,Arraial Do Cabo,BR,71.73,-22.97,-42.03,Pousada Porto Praia
31,31,Cururupu,BR,75.27,-1.83,-44.87,Pousada Través'cia
35,35,Cidreira,BR,63.79,-30.18,-50.21,Novomar Pousada e Turismo Ltda
67,67,Gilbues,BR,72.63,-9.83,-45.34,Hotel Camapuã


In [29]:
city_choice_Gilbues = vacation_data_br.loc[vacation_data_br['City'] == 'Gilbues']
city_choice_Arraial = vacation_data_br.loc[vacation_data_br['City'] == 'Arraial Do Cabo']
city_choice_Cururupu = vacation_data_br.loc[vacation_data_br['City'] == 'Cururupu']
city_choice_Cidreira = vacation_data_br.loc[vacation_data_br['City'] == 'Cidreira']

In [30]:
city_choice_Gilbues

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
67,67,Gilbues,BR,72.63,-9.83,-45.34,Hotel Camapuã


In [31]:
city_choice_Arraial

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
28,28,Arraial Do Cabo,BR,71.73,-22.97,-42.03,Pousada Porto Praia


In [32]:
city_choice_Cururupu

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
31,31,Cururupu,BR,75.27,-1.83,-44.87,Pousada Través'cia


In [33]:
city_choice_Cidreira

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
35,35,Cidreira,BR,63.79,-30.18,-50.21,Novomar Pousada e Turismo Ltda


In [35]:
# Latitude and Longitud for city choice
# location_br = (city_choice_br.Lat, city_choice_br.Lng)
# location_ar = (city_choice_ar.Lat, city_choice_ar.Lng)
# location_cl = (city_choice_cl.Lat, city_choice_cl.Lng)
# location_co = (city_choice_co.Lat, city_choice_co.Lng)
location_Gilbues = tuple(zip(city_choice_Gilbues.Lat, city_choice_Gilbues.Lng))[0]
location_Arraial = tuple(zip(city_choice_Arraial.Lat, city_choice_Arraial.Lng))[0]
location_Cururupu = tuple(zip(city_choice_Cururupu.Lat, city_choice_Cururupu.Lng))[0]
location_Cidreira = tuple(zip(city_choice_Cidreira.Lat, city_choice_Cidreira.Lng))[0]

In [36]:
location_Gilbues

(-9.83, -45.34)

In [37]:
location_Arraial

(-22.97, -42.03)

In [38]:
location_Cururupu

(-1.83, -44.87)

In [39]:
location_Cidreira

(-30.18, -50.21)

In [40]:
waypoints = [location_Arraial, location_Cidreira]
waypoints

[(-22.97, -42.03), (-30.18, -50.21)]

In [41]:
vacation_brasil = pd.concat([city_choice_Gilbues, city_choice_Arraial, city_choice_Cururupu, city_choice_Cidreira])
vacation_brasil

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
67,67,Gilbues,BR,72.63,-9.83,-45.34,Hotel Camapuã
28,28,Arraial Do Cabo,BR,71.73,-22.97,-42.03,Pousada Porto Praia
31,31,Cururupu,BR,75.27,-1.83,-44.87,Pousada Través'cia
35,35,Cidreira,BR,63.79,-30.18,-50.21,Novomar Pousada e Turismo Ltda


In [42]:
fig = gmaps.figure()
brasil2colombia = gmaps.directions_layer(
        location_Cururupu, location_Gilbues, waypoints,
        travel_mode='DRIVING')
fig.add_layer(brasil2colombia)
fig

Figure(layout=FigureLayout(height='420px'))

In [43]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
brasil_trip = [info_box_template.format(**row) for index, row in vacation_brasil.iterrows()]

In [45]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = vacation_brasil[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

brasil2colombia = gmaps.directions_layer(
        location_Cururupu, location_Gilbues, waypoints,
        travel_mode='DRIVING', show_markers = False)

marker_layer = gmaps.marker_layer(locations, info_box_content = brasil_trip)
fig.add_layer(brasil2colombia)
fig.add_layer(marker_layer)

marker_layer.markers[0].label='D'
marker_layer.markers[1].label='B'
marker_layer.markers[2].label='A'
marker_layer.markers[3].label='C'


# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))